In [3]:
from apscheduler.schedulers.blocking import BlockingScheduler
from oandapyV20 import API
import oandapyV20.endpoints.orders as orders
from oandapyV20.contrib.requests import MarketOrderRequest
from oanda_candles import Pair, Gran, CandleClient
from oandapyV20.endpoints.instruments import InstrumentsCandles
from oandapyV20.contrib.requests import TakeProfitDetails, StopLossDetails
import talib
import pandas_ta as ta
import numpy as np
import pandas as pd


In [21]:
def signal_generator(df):
    vwap = df['VWAP']
    rsi = df['RSI'].dropna()

    close_price = df['Close']
    upper_bb = df['UpperBB']
    lower_bb = df['LowerBB']
    average_rsi = rsi.mean()
    bullish = True
    bearish = True
    sell_count = 0
    buy_count = 0
    #print(rsi.values)
    vwap_threshold = 0.0004
    for i in range(len(df)):
        if max(df['Open'][i], df['Close'][i]) >= (1 + vwap_threshold) * df['VWAP'][i]:
            bearish = False
        if min(df['Open'][i], df['Close'][i]) <= (1 - vwap_threshold) * df['VWAP'][i]:
            bullish = False

    if bullish and bearish:
        vwap_signal =  "Neutral"  # Neither Bullish nor Bearish
    elif bullish:
        vwap_signal =  "Bullish"
        buy_count += 1
    elif bearish:
        vwap_signal =  "Bearish"
        sell_count += 1
    else:
        vwap_signal =  "Neutral"

    if average_rsi > 70:
        rsi_signal = "Overbought"
        sell_count += 1
    elif average_rsi < 30:
        rsi_signal = "Oversold"
        buy_count += 1
    else:
        rsi_signal = "Neutral"
        
    bb_threshold = 0.2
    overbought_signal = (df['Close'] > upper_bb).any()
    oversold_signal = (df['Close'] < lower_bb).any()

    if overbought_signal:
        bb_signal = "Overbought"
        sell_count += 1
    elif oversold_signal:
        bb_signal = "Oversold"
        buy_count += 1
    else:
        bb_signal = "Neutral"

    if buy_count >= 2:
        #print(vwap_signal, rsi_signal, bb_signal )
        return "Buy"
    elif sell_count >= 2:
        #print(vwap_signal, rsi_signal, bb_signal )
        return "Sell"
    else:
        return "-"



### 3 - Connect to the market and execute trades

In [5]:
access_token= "cd7e21b015cb734770f5fcfea9da9c9f-49a95de76ed5cab29ee0d33cee5f004d"
accountID = "101-001-27332384-001" #your account ID here
list_of_currency = [Pair.EUR_USD, Pair.EUR_GBP, Pair.USD_JPY, Pair.AUD_USD, Pair.USD_CAD, Pair.CAD_JPY, Pair.NZD_USD]
def get_candles(n, currency):
    client = API(access_token)
    request = InstrumentsCandles(instrument=currency, params={"granularity": Gran.M15, "count": n})
    response = client.request(request)
    candles = response['candles']
    return candles

candles = get_candles(3, Pair.EUR_USD)
candles


[{'complete': True,
  'volume': 308,
  'time': '2023-11-10T21:15:00.000000000Z',
  'mid': {'o': '1.06856', 'h': '1.06865', 'l': '1.06838', 'c': '1.06848'}},
 {'complete': True,
  'volume': 373,
  'time': '2023-11-10T21:30:00.000000000Z',
  'mid': {'o': '1.06848', 'h': '1.06863', 'l': '1.06832', 'c': '1.06836'}},
 {'complete': True,
  'volume': 408,
  'time': '2023-11-10T21:45:00.000000000Z',
  'mid': {'o': '1.06834', 'h': '1.06854', 'l': '1.06826', 'c': '1.06847'}}]

In [6]:
def calculate_vwap(data):
    data['VWAP'] = (((data['High'] + data['Low'] + data['Close']) / 3) * data['Volume']).cumsum() / data['Volume'].cumsum()

def calculate_bollinger_bands(data, std_dev=2):
    data['SMA'] = data['Close'].mean()
    std_dev_values = data['Close'].std()
    data['UpperBB'] = data['SMA'] + (std_dev * std_dev_values)
    data['LowerBB'] = data['SMA'] - (std_dev * std_dev_values)
    
def calculate_rsi(data):
    data['RSI'] = talib.RSI(data['Close'])

In [7]:
def trading_job(currency, string_currency):
    candles = get_candles(15, currency)
    dfstream = pd.DataFrame(columns=['Open','Close','High','Low', 'Volume'])
    i=0

    for candle in candles:
        dfstream.loc[i, 'Open'] = float(candle['mid']['o'])
        dfstream.loc[i, 'Close'] = float(candle['mid']['c'])
        dfstream.loc[i, 'High'] = float(candle['mid']['h'])
        dfstream.loc[i, 'Low'] = float(candle['mid']['l'])
        dfstream.loc[i, 'Volume'] = float(candle['volume'])
        i += 1

    dfstream['Open'] = dfstream['Open'].astype(float)
    dfstream['Close'] = dfstream['Close'].astype(float)
    dfstream['High'] = dfstream['High'].astype(float)
    dfstream['Low'] = dfstream['Low'].astype(float)
    dfstream['Volume'] = dfstream['Volume'].astype(float)
    calculate_vwap(dfstream)
    calculate_bollinger_bands(dfstream)
    calculate_rsi(dfstream)

    #print(dfstream)
    #print(dfstream)
    signal = signal_generator(dfstream)#
    # EXECUTING ORDERS
    print("signal: " + str(signal))
    client = API(access_token)

    SLTPRatio = 2.
    previous_candleR = abs(dfstream['High'].iloc[-2]-dfstream['Low'].iloc[-2])

    SLBuy = float(str(candle['mid']['o']))-previous_candleR
    SLSell = float(str(candle['mid']['o']))+previous_candleR

    TPBuy = float(str(candle['mid']['o']))+previous_candleR*SLTPRatio
    TPSell = float(str(candle['mid']['o']))-previous_candleR*SLTPRatio

    #print(dfstream.iloc[:-1,:])
    #print(TPBuy, "  ", SLBuy, "  ", TPSell, "  ", SLSell)
    #signal = 2
    #Sell
    if signal == "Sell":
        mo = MarketOrderRequest(instrument=string_currency, units=-2000, takeProfitOnFill=TakeProfitDetails(price=TPSell).data, stopLossOnFill=StopLossDetails(price=SLSell).data)
        r = orders.OrderCreate(accountID, data=mo.data)
        rv = client.request(r)
        print(rv)
    #Buy
    elif signal == "Buy":
        mo = MarketOrderRequest(instrument=string_currency, units=2000, takeProfitOnFill=TakeProfitDetails(price=TPBuy).data, stopLossOnFill=StopLossDetails(price=SLBuy).data)
        r = orders.OrderCreate(accountID, data=mo.data)
        rv = client.request(r)
        print(rv)

In [23]:
list_of_currency = [Pair.EUR_USD, Pair.EUR_GBP, Pair.USD_JPY, Pair.AUD_USD, Pair.USD_CAD, Pair.CAD_JPY, Pair.NZD_USD]
list_of_str_currency = ["EUR_USD", "EUR_GBP", "USD_JPY", "AUD_USD","USD_CAD","CAD_JPY","NZD_USD"]

# for i in range (len(list_of_str_currency)):
#     trading_job(list_of_currency[i],list_of_str_currency[i])


scheduler = BlockingScheduler()
for i in range (len(list_of_str_currency)):
    scheduler.add_job(trading_job, 'cron', args = [list_of_currency[i],list_of_str_currency[i]], day_of_week='mon-fri', hour='00-23', minute='1,11,21,31,41,51', start_date='2022-01-12 12:00:00')
scheduler.start()
#ADD RANDOM

signal: -signal: -
signal: -
signal: -

signal: -
signal: -
signal: -
signal: -
signal: -
signal: -
signal: -
signal: -
signal: -
signal: -
signal: -
signal: -
signal: -
signal: -
signal: -
signal: -
signal: -
signal: -signal: -

signal: -
signal: -
signal: -
signal: -
signal: -
signal: -signal: -

signal: -
signal: -
signal: -
signal: -
signal: -
signal: -signal: -

signal: -
signal: -
signal: -
signal: -
signal: -
signal: -signal: -
signal: -
signal: -

signal: -
signal: -
signal: -
signal: -signal: -

signal: -
signal: -
signal: -
signal: -
signal: -


In [22]:
da= pd.read_csv("EURUSD_Candlestick_5_M_ASK_30.09.2019-30.09.2022.csv")
da = da[:10000]
signals = []

for i in range(0, len(da)):
    block = da.iloc[i:i+15].copy()
    calculate_vwap(block)
    calculate_bollinger_bands(block)
    calculate_rsi(block)
    block = block.reset_index(drop=True)
    if not block.empty:
        signal = signal_generator(block)
        signals.append(signal)
x = pd.Series(signals)
x.value_counts()


-       9209
Buy      397
Sell     394
Name: count, dtype: int64